In [2]:
import requests
import pandas as pd
import joblib
import datetime
import os
from ipywidgets import widgets, interact

# load trained model
model_path = "models/xgboost_best_model.pkl"
model = joblib.load(model_path)

# create logs directory if it doesn't exist
log_dir = "logs"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# function to log errors
def log_error(error_message):
    """logs api errors into a file for debugging"""
    log_file = os.path.join(log_dir, "error_log.txt")
    with open(log_file, "a") as f:
        f.write(f"[error] {datetime.datetime.now()} - {error_message}\n")
    print(f"error logged: {error_message}")

# function to log predictions
def log_prediction(date, decision, confidence):
    """logs model predictions for tracking"""
    log_file = os.path.join(log_dir, "predictions_log.txt")
    with open(log_file, "a") as f:
        f.write(f"[prediction] {datetime.datetime.now()} | date: {date} | decision: {decision} | confidence: {confidence:.2f}%\n")
    print(f"prediction logged: {date}, {decision}, {confidence:.2f}%")

# function to fetch a 7-day weather forecast from open-meteo api
def fetch_weather_forecast():
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": 28.40057,  
        "longitude": -80.611755,
        "daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum", 
                  "windspeed_10m_max", "windgusts_10m_max", "winddirection_10m_dominant", 
                  "cloudcover_mean", "pressure_msl_mean"],
        "timezone": "GMT",
        "forecast_days": 7  
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  
        return response.json()
    
    except requests.exceptions.RequestException as e:
        log_error(f"api request failed: {e}")
        return None  

# fetch forecast data once
weather_data = fetch_weather_forecast()
if weather_data is None:
    raise SystemExit("unable to fetch weather data. check logs for details.")

# extract available dates for selection
available_dates = [datetime.datetime.strptime(d, "%Y-%m-%d").date() for d in weather_data["daily"]["time"][:7]]

# function to get weather data for a selected date
def get_weather_by_date(selected_date, weather_data):
    date_str = selected_date.strftime("%Y-%m-%d")
    
    try:
        date_index = weather_data["daily"]["time"].index(date_str)
        
        temperature_c = (weather_data["daily"]["temperature_2m_max"][date_index] + 
                         weather_data["daily"]["temperature_2m_min"][date_index]) / 2
        temperature_f = (temperature_c * 9/5) + 32  
        precipitation_mm = weather_data["daily"]["precipitation_sum"][date_index]
        precipitation_inches = precipitation_mm / 25.4  

        features = pd.DataFrame([{
            "temperature_2m": temperature_f,
            "relative_humidity_2m": 60.0,  
            "precipitation": precipitation_inches,
            "windspeed_10m": weather_data["daily"]["windspeed_10m_max"][date_index],
            "windgusts_10m": weather_data["daily"]["windgusts_10m_max"][date_index],
            "winddirection_10m": weather_data["daily"]["winddirection_10m_dominant"][date_index],
            "cloudcover": weather_data["daily"]["cloudcover_mean"][date_index],
            "pressure_msl": weather_data["daily"]["pressure_msl_mean"][date_index]
        }])
        
        return features
    except ValueError:
        log_error(f"error: weather data for {date_str} is not available.")
        print(f"error: weather data for {date_str} is not available.")
        return None

# function to run prediction
def predict_launch(selected_date):
    weather_features = get_weather_by_date(selected_date, weather_data)
    
    if weather_features is not None:
        prediction = model.predict(weather_features)[0]
        confidence = model.predict_proba(weather_features)[0][int(prediction)] * 100
        decision = "GO" if prediction == 1 else "NO-GO"

        # log the prediction
        log_prediction(selected_date.strftime('%Y-%m-%d'), decision, confidence)
        
        print(f"date: {selected_date.strftime('%Y-%m-%d')}")
        print(f"launch decision: {decision}")
        print(f"confidence score: {confidence:.2f}%")

# updated date picker with only valid dates (7 days max)
date_picker = widgets.SelectionSlider(
    options=available_dates,  
    description="select date:",
    continuous_update=False
)

interact(predict_launch, selected_date=date_picker)



interactive(children=(SelectionSlider(continuous_update=False, description='select date:', options=(datetime.d…

<function __main__.predict_launch(selected_date)>

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# load and display feature importance
plt.figure(figsize=(12, 6))
img = mpimg.imread("visualizations/feature_importance_comparison.png")
plt.imshow(img)
plt.axis("off")
plt.title("Feature Importance Comparison")
plt.show()

# load and display confusion matrix
plt.figure(figsize=(8, 6))
img = mpimg.imread("visualizations/confusion_matrix.png")
plt.imshow(img)
plt.axis("off")
plt.title("Confusion Matrix")
plt.show()

# load and display threshold comparison
plt.figure(figsize=(12, 5))
img = mpimg.imread("visualizations/threshold_comparison.png")
plt.imshow(img)
plt.axis("off")
plt.title("Threshold Comparison")
plt.show()
